In [7]:
from datetime import datetime
import pandas as pd
import numpy as np
import json
import re
import string
import nltk

In [36]:
df = pd.read_json("bq_2018_top5SIC.json", lines = True)
df.head()

,accessionNumber,filingDate,reportingDate,financialEntity,htmlFile,coDescription,CIK,name,countryinc,cityma,SIC,SIC_desc
0,0001441816-18-000028,2018-03-30 20:12:23 UTC,2018-02-01,financialEntities/params;cik=1441816,https://www.sec.gov/Archives/edgar/data/144181...,Item 1. BusinessOverviewMongoDB is the leading...,1441816,"MONGODB, INC.",US,NEW YORK,7372,Prepackaged Software (mass reproduction of sof...
1,0001108524-18-000011,2018-03-09 22:01:46 UTC,2018-02-01,financialEntities/params;cik=1108524,https://www.sec.gov/Archives/edgar/data/110852...,ITEM 1. BUSINESSOverviewSalesforce is a global...,1108524,SALESFORCE COM INC,US,SAN FRANCISCO,7372,Prepackaged Software (mass reproduction of sof...
2,0001564590-18-006986,2018-03-28 21:27:30 UTC,2018-02-01,financialEntities/params;cik=1385867,https://www.sec.gov/Archives/edgar/data/138586...,"Item 1.Business1<p style=""margin-bottom:0pt;ma...",1385867,COUPA SOFTWARE INC,US,SAN MATEO,7372,Prepackaged Software (mass reproduction of sof...
3,0001353283-18-000004,2018-03-30 21:21:46 UTC,2018-02-01,financialEntities/params;cik=1353283,https://www.sec.gov/Archives/edgar/data/135328...,Item 1. BusinessOverviewSplunk provides innov...,1353283,SPLUNK INC,NaN,SAN FRANCISCO,7372,Prepackaged Software (mass reproduction of sof...
4,0001660134-18-000007,2018-03-12 20:45:43 UTC,2018-02-01,financialEntities/params;cik=1660134,https://www.sec.gov/Archives/edgar/data/166013...,Item 1. BusinessOverview Okta is the leading i...,1660134,"OKTA, INC.",US,SAN FRANCISCO,7372,Prepackaged Software (mass reproduction of sof...


In [9]:
#strip any left over html code
def clean_data_fn(insrt_data):
    
    clean_data = []
    
    for idx, ele in insrt_data.iterrows():
        if "https://www.sec.gov/Archives/edgar/data/" in ele["coDescription"]:
            pass
        else:
            clean_txt = re.compile('<.*?>')
            desc = re.sub(clean_txt,'',ele["coDescription"]).replace(u'\xa0', u' ').replace("   ", "").replace("'", "").replace('"','')
            if re.search('<', desc):
                pos = re.search('<', desc).start()
            desc = desc[:pos].lower()
            if (desc.find("business") >= 20): # didnt find it in the first 20 characters then look for next
                desc = desc[6 : ( desc.rfind("<") )] # remove the "Item 1." stuff only
            else: # found "business", remove everything before it
                desc =  desc[( desc.find("business") + 8 ) : ( desc.rfind("<") ) ]
            if (desc.find("overview") <= 20): # didnt find it in the first 20 characters then look for next
                desc =  desc[( desc.find("overview") + 8 ) :]
            # remove leading white space and periods
            desc = re.sub(r"^\.", "", desc).strip()            
            new_data = ele.copy()
            new_data["coDescription"] = desc
            # remove any filings with a description less than 250 characters (not enough information for us)
            if len(desc)<250:
                pass
            else:
                clean_data.append(new_data)
                
    return(pd.DataFrame(clean_data))

non_html_data = clean_data_fn(df)#.rename(columns = {"financialEntity":"CIK"})
non_html_data["CIK"] = non_html_data["CIK"].astype(int)
non_html_data.head()

,accessionNumber,filingDate,reportingDate,financialEntity,htmlFile,coDescription,CIK,name,countryinc,cityma,SIC,SIC_desc
0,0001441816-18-000028,2018-03-30 20:12:23 UTC,2018-02-01,financialEntities/params;cik=1441816,https://www.sec.gov/Archives/edgar/data/144181...,"mongodb is the leading modern, general purpose...",1441816,"MONGODB, INC.",US,NEW YORK,7372,Prepackaged Software (mass reproduction of sof...
1,0001108524-18-000011,2018-03-09 22:01:46 UTC,2018-02-01,financialEntities/params;cik=1108524,https://www.sec.gov/Archives/edgar/data/110852...,salesforce is a global leader in customer rela...,1108524,SALESFORCE COM INC,US,SAN FRANCISCO,7372,Prepackaged Software (mass reproduction of sof...
3,0001353283-18-000004,2018-03-30 21:21:46 UTC,2018-02-01,financialEntities/params;cik=1353283,https://www.sec.gov/Archives/edgar/data/135328...,splunk provides innovative software solutions ...,1353283,SPLUNK INC,NaN,SAN FRANCISCO,7372,Prepackaged Software (mass reproduction of sof...
4,0001660134-18-000007,2018-03-12 20:45:43 UTC,2018-02-01,financialEntities/params;cik=1660134,https://www.sec.gov/Archives/edgar/data/166013...,okta is the leading independent provider of id...,1660134,"OKTA, INC.",US,SAN FRANCISCO,7372,Prepackaged Software (mass reproduction of sof...
5,0001564590-18-007164,2018-03-29 21:34:05 UTC,2018-02-01,financialEntities/params;cik=1393052,https://www.sec.gov/Archives/edgar/data/139305...,veeva is a leading provider of industry cloud ...,1393052,VEEVA SYSTEMS INC,NaN,PLEASANTON,7372,Prepackaged Software (mass reproduction of sof...


In [17]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
# nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
    
def lemmatize_sentence(sentence):
    lemmatized_output = [lemmatizer.lemmatize(w) for w in word_tokenize(sentence)]
#     lemmatized_output = [(lemmatizer.lemmatize(w) if nltk.pos_tag([w])[0][1][0].upper() == "N" else "") for w in word_tokenize(sentence)]
#     noun_only = ""
#     for word in lemmatized_output:
#         if(nltk.pos_tag([word])[0][1][0].upper() == "N"):
#             noun_only = noun_only + word + " "    
#     return noun_only
    return " ".join(lemmatized_output)

lemma_desc = non_html_data["coDescription"].apply(lemmatize_sentence)
non_html_data["coDescription_lemmatized"] = lemma_desc
non_html_data= non_html_data.reset_index()
non_html_data["coDescription_lemmatized"].head()

0    mongodb is the leading modern , general purpos...
1    salesforce is a global leader in customer rela...
3    splunk provides innovative software solution t...
4    okta is the leading independent provider of id...
5    veeva is a leading provider of industry cloud ...
Name: coDescription_lemmatized, dtype: object

In [18]:
from nltk.corpus import stopwords
nltk.download('stopwords')

# remove all numbers so they don't show up as dimensions
def remove_nums(x):
    text = x.lower()
    text = re.sub(r'\d+', '', text)
    return text

def remove_stopwords(x):
    stop_words = set(stopwords.words('english'))

    word_tokens = word_tokenize(x)

    filtered_sentence = ' '.join([w for w in word_tokens if not w.lower() in stop_words and w.isalnum()])

    return(filtered_sentence)

rm_num_stopwords = non_html_data["coDescription_lemmatized"].apply(remove_nums).apply(remove_stopwords)
non_html_data["coDescription_stopwords"] = rm_num_stopwords
rm_num_stopwords.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maryx\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    mongodb leading modern general purpose databas...
1    salesforce global leader customer relationship...
3    splunk provides innovative software solution e...
4    okta leading independent provider identity ent...
5    veeva leading provider industry cloud solution...
Name: coDescription_lemmatized, dtype: object

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

doc = CountVectorizer(ngram_range=(1,4))
word_count=doc.fit_transform(rm_num_stopwords)

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=doc.get_feature_names(),columns=["idf_weights"])
df_idf.sort_values(by=['idf_weights']).head(40)

tf_idf_vector=tfidf_transformer.transform(word_count)
feature_names = doc.get_feature_names()

C:\Users\maryx\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\maryx\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [33]:
def getImportantWords(company_index):
    first_document_vector=tf_idf_vector[company_index]
    temp = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
    return temp.sort_values(by=["tfidf"],ascending=False).head(45)

,tfidf
mongodb,0.389086
database,0.196785
developer,0.155608
platform,0.135699
mongodb atlas,0.128380
community server,0.121248
customer,0.118797
atlas,0.117314
server,0.109231
cloud,0.098725


In [72]:
## Get index of APPIAN, MONGODB, SALESFORCE
## APPIAN AND SALESFORCE HAVE SIMILAR RETURNS IN 2018
## APPIAN AND MONGODB HAVE DISSIMILAR RETURNS IN 2018

def getIndex(companyName):
    condition = non_html_data["name"] == companyName
    index = non_html_data.index
    return index[condition].tolist()

MONGODB = getIndex("MONGODB, INC.")[0]
APPIAN = getIndex("APPIAN CORP")[0]
SALESFORCE = getIndex("SALESFORCE COM INC")[0]

                                tfidf
mongodb                      0.389086
database                     0.196785
developer                    0.155608
platform                     0.135699
mongodb atlas                0.128380
community server             0.121248
customer                     0.118797
atlas                        0.117314
server                       0.109231
cloud                        0.098725
application                  0.095174
software                     0.080736
enterprise                   0.079853
developer productivity       0.067084
subscription                 0.066367
relational                   0.065398
relational database          0.059948
mongodb enterprise           0.057058
legacy database              0.057058
offering                     0.056910
built                        0.050712
enterprise advanced          0.049926
community                    0.047250
organization                 0.044886
mongodb enterprise advanced  0.042793
user        

In [73]:
getImportantWords(MONGODB)

,tfidf
mongodb,0.389086
database,0.196785
developer,0.155608
platform,0.135699
mongodb atlas,0.128380
community server,0.121248
customer,0.118797
atlas,0.117314
server,0.109231
cloud,0.098725


In [74]:
getImportantWords(APPIAN)

,tfidf
platform,0.202909
appian,0.181988
customer,0.133505
software,0.113454
application,0.102286
sail,0.096750
organization,0.084140
user,0.083244
kx,0.079125
interface,0.071054


In [75]:
getImportantWords(SALESFORCE)

,tfidf
cloud,0.220839
customer,0.180660
apps,0.135817
salesforce,0.128698
service,0.119417
subscription,0.095385
salesforce platform,0.091341
platform,0.088367
solution,0.071163
service offering,0.069053


In [78]:
def findDuplicates(a, b):
    d = {}
    dup = []
    for word in a:
        d[word] = 1
    for word in b:
        if word in d:
            dup.append(word)
    return dup

['platform',
 'customer',
 'software',
 'application',
 'subscription',
 'functionality',
 'cloud',
 'enterprise',
 'service',
 'revenue']

In [80]:
### Similarity between APPIAN and SALESFORCE
findDuplicates(getImportantWords(SALESFORCE).index.values, getImportantWords(APPIAN).index.values)

['platform',
 'customer',
 'software',
 'application',
 'subscription',
 'functionality',
 'cloud',
 'enterprise',
 'service',
 'revenue']

In [81]:
### Similarity between APPIAN and MONGODB
findDuplicates(getImportantWords(MONGODB).index.values, getImportantWords(APPIAN).index.values)

['platform',
 'customer',
 'software',
 'application',
 'organization',
 'user',
 'subscription',
 'cloud',
 'enterprise',
 'deployment']